# Homework 2 - Data from the Web

# Task 1 - QS World Universities ranking

### Importing libs

In [32]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt


import requests
from bs4 import BeautifulSoup

import json

## Extracting data from the website

In [72]:
default_path = 'https://www.topuniversities.com'
r = requests.get(default_path +'/sites/default/files/qs-rankings-data/357051.txt?_=1507995842896')
result_json = r.json()

result = pd.DataFrame.from_dict(result_json['data']).head(200)
result.head(10)

,cc,core_id,country,guide,logo,nid,rank_display,region,score,stars,title,url
0,US,410,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...
1,US,573,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",297282,2,North America,98.7,5,Stanford University,/universities/stanford-university
2,US,253,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294270,3,North America,98.4,5,Harvard University,/universities/harvard-university
3,US,94,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294562,4,North America,97.7,5,California Institute of Technology (Caltech),/universities/california-institute-technology-...
4,GB,95,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294561,5,Europe,95.6,5,University of Cambridge,/universities/university-cambridge
5,GB,478,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294654,6,Europe,95.3,5,University of Oxford,/universities/university-oxford
6,GB,365,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294014,7,Europe,94.6,None,UCL (University College London),/universities/ucl-university-college-london
7,GB,356,United Kingdom,"<a href=""/where-to-study/europe/united-kingdom...","<img src=""https://www.topuniversities.com/site...",294030,8,Europe,93.7,None,Imperial College London,/universities/imperial-college-london
8,US,120,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294536,9,North America,93.5,5,University of Chicago,/universities/university-chicago
9,CH,201,Switzerland,"<a href=""/where-to-study/europe/switzerland/gu...","<img src=""https://www.topuniversities.com/site...",294432,10,Europe,93.3,None,ETH Zurich - Swiss Federal Institute of Techno...,/universities/eth-zurich-swiss-federal-institu...


In [36]:
for index, row in result.iterrows():
        r_ = requests.get(default_path + result.iloc[index]['url'])
        soup = BeautifulSoup(r_.text, 'html.parser')
        university = result.iloc[index]['title']
        try:
            total_faculty = soup.find('div', class_='total faculty')
            result.loc[index, 'Staff total number'] = int(total_faculty.find('div', class_='number').text.split()[0].replace(',', ''))
        except:
            print(university + ' has no values for TOTAL_STAFF') 
        try:
            total_student = soup.find('div', class_='total student')
            result.loc[index, 'Students total number'] = int(total_student.find('div', class_='number').text.split()[0].replace(',', ''))
        except:
            print(university + ' has no values for TOTAL_STUDENTS')
        try:
            inter_faculty = soup.find('div', class_='inter faculty')
            result.loc[index, 'International Staff number'] = int(inter_faculty.find('div', class_='number').text.split()[0].replace(',', ''))
        except:
            print(university + ' has no values for INTERNATIONAL_STAFF')
        try:
            inter_student = soup.find('div', class_='total inter')
            result.loc[index, 'International Students number'] = int(inter_student.find('div', class_='number').text.split()[0].replace(',', ''))
        except:
            print(university + ' has no values for INTERNATIONAL_STUDENTS')
            print(default_path + result.iloc[index]['url'])

New York University (NYU) has no values for TOTAL_STAFF
New York University (NYU) has no values for TOTAL_STUDENTS
New York University (NYU) has no values for INTERNATIONAL_STAFF
New York University (NYU) has no values for INTERNATIONAL_STUDENTS
https://www.topuniversities.com/universities/new-york-university-nyu
Indian Institute of Science (IISc) Bangalore has no values for INTERNATIONAL_STAFF


### Removing useless columns

In [37]:
result.fillna(0)
useless_columns = ['core_id','guide','nid','url','logo']
for column_name in useless_columns:
    del result[column_name]

In [38]:
df = result.copy()
df.head()

,cc,country,rank_display,region,score,stars,title,Staff total number,Students total number,International Staff number,International Students number
0,US,United States,1,North America,100,6,Massachusetts Institute of Technology (MIT),2982.0,11067.0,1679.0,3717.0
1,US,United States,2,North America,98.7,5,Stanford University,4285.0,15878.0,2042.0,3611.0
2,US,United States,3,North America,98.4,5,Harvard University,4350.0,22429.0,1311.0,5266.0
3,US,United States,4,North America,97.7,5,California Institute of Technology (Caltech),953.0,2255.0,350.0,647.0
4,GB,United Kingdom,5,Europe,95.6,5,University of Cambridge,5490.0,18770.0,2278.0,6699.0


## Extracting ratios

In [7]:
df['Ratio Staff/Student']=df['Staff total number']/df['Students total number']
df['Ratio International/National Students']=df['International Students number']/df['Students total number']

### Sorting dataframes by ratios

In [8]:
df_international = df.sort_values('Ratio International/National Students', ascending=0)
df_staffstud = df.sort_values('Ratio Staff/Student', ascending=0)

In [9]:
df_international.head(3)

,cc,country,rank_display,region,score,stars,title,Staff total number,Students total number,International Staff number,International Students number,Ratio Staff/Student,Ratio International/National Students
34,GB,United Kingdom,35,Europe,81.8,None,London School of Economics and Political Scien...,1088.0,9760.0,687.0,6748.0,0.111475,0.691393
11,CH,Switzerland,12,Europe,91.2,None,Ecole Polytechnique Fédérale de Lausanne (EPFL),1695.0,10343.0,1300.0,5896.0,0.163879,0.570047
7,GB,United Kingdom,8,Europe,93.7,None,Imperial College London,3930.0,16090.0,2071.0,8746.0,0.244251,0.543567


In [10]:
df_staffstud.head(3)

,cc,country,rank_display,region,score,stars,title,Staff total number,Students total number,International Staff number,International Students number,Ratio Staff/Student,Ratio International/National Students
3,US,United States,4,North America,97.7,5,California Institute of Technology (Caltech),953.0,2255.0,350.0,647.0,0.422616,0.286918
15,US,United States,16,North America,90.4,5,Yale University,4940.0,12402.0,1708.0,2469.0,0.398323,0.199081
5,GB,United Kingdom,6,Europe,95.3,5,University of Oxford,6750.0,19720.0,2964.0,7353.0,0.342292,0.372870


### Deleting useless columns before grouping by Country and Region

In [11]:

df_international_by_country_or_region = df_international.copy()
other_useless_columns = ['rank_display','score','stars','cc','Ratio International/National Students','Ratio Staff/Student','title']
for useless_column in other_useless_columns:
    del df_international_by_country_or_region[useless_column]
df_international_by_country_or_region.head(5)

,country,region,Staff total number,Students total number,International Staff number,International Students number
34,United Kingdom,Europe,1088.0,9760.0,687.0,6748.0
11,Switzerland,Europe,1695.0,10343.0,1300.0,5896.0
7,United Kingdom,Europe,3930.0,16090.0,2071.0,8746.0
198,Netherlands,Europe,1277.0,16385.0,502.0,8234.0
47,United States,North America,1342.0,13356.0,425.0,6385.0


## Grouping by country and Region

In [12]:
df_international_by_country = df_international_by_country_or_region.groupby(['country']).sum()
df_international_by_region = df_international_by_country_or_region.groupby(['region']).sum()

### Producing ratios by country

In [13]:
df_international_by_country['Ratio Staff/Student'] = df_international_by_country['Staff total number']/df_international_by_country['Students total number']
df_international_by_country['Ratio International/National Students'] = df_international_by_country['International Students number']/df_international_by_country['Students total number']

In [14]:
df_international_by_country_staffstudent = df_international_by_country.copy()
df_international_by_country_staffstudent.sort_values('Ratio Staff/Student', ascending=0).head(5)

,Staff total number,Students total number,International Staff number,International Students number,Ratio Staff/Student,Ratio International/National Students
country,,,,,,
Russia,6709.0,30233.0,373.0,5098.0,0.221910,0.168624
Denmark,11916.0,67223.0,3904.0,9543.0,0.177261,0.141960
Saudi Arabia,1062.0,6040.0,665.0,989.0,0.175828,0.163742
Singapore,9444.0,58466.0,6079.0,16168.0,0.161530,0.276537
Malaysia,2755.0,17902.0,655.0,3476.0,0.153893,0.194168


In [15]:
df_international_by_country_intstudent = df_international_by_country.copy()
df_international_by_country_intstudent.sort_values('Ratio International/National Students', ascending=0).head(5)

,Staff total number,Students total number,International Staff number,International Students number,Ratio Staff/Student,Ratio International/National Students
country,,,,,,
Australia,22034.0,301994.0,11382.0,106359.0,0.072962,0.352189
United Kingdom,79934.0,583621.0,30216.0,199426.0,0.136962,0.341705
Hong Kong,10166.0,78838.0,6296.0,24499.0,0.128948,0.310751
Austria,4117.0,63446.0,1572.0,19667.0,0.064890,0.309980
Switzerland,15323.0,109112.0,9208.0,32995.0,0.140434,0.302396


### Producing ratios by region

In [16]:
df_international_by_region['Ratio Staff/Student'] = df_international_by_region['Staff total number']/df_international_by_region['Students total number']
df_international_by_region['Ratio International/National Students'] = df_international_by_region['International Students number']/df_international_by_region['Students total number']

In [17]:
df_international_by_region_staffstudent = df_international_by_region.copy()
df_international_by_region_staffstudent.sort_values('Ratio Staff/Student', ascending=0).head(5)

,Staff total number,Students total number,International Staff number,International Students number,Ratio Staff/Student,Ratio International/National Students
region,,,,,,
Asia,106734.0,807003.0,25462.0,110100.0,0.132260,0.136431
North America,182123.0,1546353.0,43836.0,292116.0,0.117776,0.188906
Europe,218358.0,1957251.0,67598.0,449364.0,0.111564,0.229589
Latin America,45382.0,435750.0,5648.0,36871.0,0.104147,0.084615
Africa,1733.0,19593.0,379.0,3325.0,0.088450,0.169703


In [18]:
df_international_by_region_intstudent = df_international_by_region.copy()
df_international_by_region_intstudent.sort_values('Ratio International/National Students', ascending=0).head(5)

,Staff total number,Students total number,International Staff number,International Students number,Ratio Staff/Student,Ratio International/National Students
region,,,,,,
Oceania,25347.0,350167.0,12786.0,118798.0,0.072385,0.339261
Europe,218358.0,1957251.0,67598.0,449364.0,0.111564,0.229589
North America,182123.0,1546353.0,43836.0,292116.0,0.117776,0.188906
Africa,1733.0,19593.0,379.0,3325.0,0.088450,0.169703
Asia,106734.0,807003.0,25462.0,110100.0,0.132260,0.136431


## Plots and Discussion

In [19]:
#TODO: Plot your data using bar charts and describe briefly what you observed.

# Task 2 - Times Higher Education ranking

## Extracting data from the website

In [80]:
#DUMMY EXTRACTION CODE

URL2 = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/'\
                +'world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
r2 = requests.get(URL2)
soupp = BeautifulSoup(r2.content,'html.parser')
rank_json = json.loads(r2.text)
df_times = pd.DataFrame()
df_times = rank_df.from_dict(rank_json['data']).head(200)
#rank_df.stars
#rank_df.drop(['logo', 'stars', 'nid','cc', 'score'], axis=1, inplace=True)
#rank_df.set_index('core_id', inplace=True)

print(df_times.dtypes)
df_times.head()

aliases                              object
location                             object
member_level                         object
name                                 object
nid                                   int64
rank                                 object
rank_order                           object
record_type                          object
scores_citations                     object
scores_citations_rank                object
scores_industry_income               object
scores_industry_income_rank          object
scores_international_outlook         object
scores_international_outlook_rank    object
scores_overall                       object
scores_overall_rank                  object
scores_research                      object
scores_research_rank                 object
scores_teaching                      object
scores_teaching_rank                 object
stats_female_male_ratio              object
stats_number_students                object
stats_pc_intl_students          

,aliases,location,member_level,name,nid,rank,rank_order,record_type,scores_citations,scores_citations_rank,...,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,...,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,...,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,...,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,...,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,...,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...


In [81]:
df_times.fillna(0)
useless_columns = ['member_level','nid','rank_order','record_type','scores_citations', 'scores_citations_rank', 'scores_industry_income',
                  'scores_industry_income_rank','scores_international_outlook','scores_international_outlook_rank','scores_overall',
                  'scores_overall_rank','scores_research','scores_research_rank','scores_teaching', 'scores_teaching_rank',
                  'stats_female_male_ratio','stats_number_students','stats_pc_intl_students','stats_student_staff_ratio', 'subjects_offered', 'url']
for column_name in useless_columns:
    del df_times[column_name]
df_times.head()

,aliases,location,name,rank
0,University of Oxford,United Kingdom,University of Oxford,1
1,University of Cambridge,United Kingdom,University of Cambridge,2
2,California Institute of Technology caltech,United States,California Institute of Technology,=3
3,Stanford University,United States,Stanford University,=3
4,Massachusetts Institute of Technology,United States,Massachusetts Institute of Technology,5


## Analysis

## Plots and Discussion

# Task 3 - Merging the dataframes

Steps - for explanation:
- see what happens if equality: e.g. problem for MIT
- calculate the cross product of the two dfs, i.e. combine every possible two univerities
- calculate a score for similarity between title of df and name of df_times
- using difflib library (see https://docs.python.org/2/library/difflib.html) - calculates matches by finding longest common subsequence and then recursivly looking for matches left and right of the occurence of this sequence (gestalt pattern matching introduced by Ratcliff and Obershelp in the 1980's)
- ratio of similarity in [0,1] calculated by  2.0 * M / T where M is nr of matches and T sum of the lengths of the strings
- keep only those rows where similarity is above cetain threshold (0.87)
- empirically find good value for threshold: take a look at those rows where score is strictly less than 1 but still above threshold (show code!!!), if too many wrong matches: higher threshold, as soon as reasonable keep, and remove some mistakes manually, list all exceptions
- to be sure that we did not miss anything also manually check for rows with similarity between 0.5 and 0.87 (should not be same uni), below that we considered it not possible to be same place

In [132]:
from difflib import SequenceMatcher

df_cartesian = pd.merge(df.assign(key=0), df_times.assign(key=0), on='key').drop('key', axis=1)

def similar(row):
    return SequenceMatcher(None, row['title'],row['name']).ratio()

df_cartesian['similar'] = df_cartesian.apply(similar,axis=1)
df_merged = df_cartesian[df_cartesian.similar>=0.87]
df_merged.head()

,cc,country,rank_display,region,score,stars,title,Staff total number,Students total number,International Staff number,International Students number,aliases,location,name,rank,similar
4,US,United States,1,North America,100,6,Massachusetts Institute of Technology (MIT),2982.0,11067.0,1679.0,3717.0,Massachusetts Institute of Technology,United States,Massachusetts Institute of Technology,5,0.925000
203,US,United States,2,North America,98.7,5,Stanford University,4285.0,15878.0,2042.0,3611.0,Stanford University,United States,Stanford University,=3,1.000000
405,US,United States,3,North America,98.4,5,Harvard University,4350.0,22429.0,1311.0,5266.0,Harvard University,United States,Harvard University,6,1.000000
602,US,United States,4,North America,97.7,5,California Institute of Technology (Caltech),953.0,2255.0,350.0,647.0,California Institute of Technology caltech,United States,California Institute of Technology,=3,0.871795
801,GB,United Kingdom,5,Europe,95.6,5,University of Cambridge,5490.0,18770.0,2278.0,6699.0,University of Cambridge,United Kingdom,University of Cambridge,2,1.000000


In [135]:
#what do we have to remove manually?
df_merged[df_cartesian.similar!=1]
#TODO display unis with more than one match!!!

#TODO delete those manually:
#Chinese University of Hong Kong, The University of Hong Kong
#City University of Hong Kong, The University of Hong Kong - 2x
# city - chinese
#Northwestern University, Northeastern University
#Tokyo Institute of Technology, KTH Royal Institute of Technology
#University of California, Davis, University of California, Irvine - 2x
#Lancaster University, McMaster University - 2x
#University of Bergen, University of Bern
#University of Bern, University of Bonn
#University of Rochester, University of Manchester

/home/sarah/anaconda2/envs/ada/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


,cc,country,rank_display,region,score,stars,title,Staff total number,Students total number,International Staff number,International Students number,aliases,location,name,rank,similar
4,US,United States,1,North America,100,6,Massachusetts Institute of Technology (MIT),2982.0,11067.0,1679.0,3717.0,Massachusetts Institute of Technology,United States,Massachusetts Institute of Technology,5,0.925000
602,US,United States,4,North America,97.7,5,California Institute of Technology (Caltech),953.0,2255.0,350.0,647.0,California Institute of Technology caltech,United States,California Institute of Technology,=3,0.871795
1215,GB,United Kingdom,7,Europe,94.6,None,UCL (University College London),6345.0,31080.0,2554.0,14854.0,University College London ucl,United Kingdom,University College London,16,0.892857
1809,CH,Switzerland,10,Europe,93.3,None,ETH Zurich - Swiss Federal Institute of Techno...,2477.0,19815.0,1886.0,7563.0,ETH Zurich – Swiss Federal Institute of Techno...,Switzerland,ETH Zurich – Swiss Federal Institute of Techno...,=10,0.915888
2051,SG,Singapore,11,Asia,92.2,6,"Nanyang Technological University, Singapore (NTU)",4338.0,25738.0,2993.0,7251.0,"Nanyang Technological University, Singapore N...",Singapore,"Nanyang Technological University, Singapore",52,0.934783
2237,CH,Switzerland,12,Europe,91.2,None,Ecole Polytechnique Fédérale de Lausanne (EPFL),1695.0,10343.0,1300.0,5896.0,École Polytechnique Fédérale de Lausanne,Switzerland,École Polytechnique Fédérale de Lausanne,=38,0.896552
2821,SG,Singapore,15,Asia,90.5,None,National University of Singapore (NUS),5106.0,32728.0,3086.0,8917.0,National University of Singapore nus,Singapore,National University of Singapore,=22,0.914286
3847,AU,Australia,20,Oceania,87.1,,The Australian National University,1600.0,14442.0,927.0,5551.0,Australian National University anu,Australia,Australian National University,48,0.937500
4426,GB,United Kingdom,=23,Europe,86.9,None,The University of Edinburgh,4075.0,28040.0,1553.0,10551.0,University of Edinburgh,United Kingdom,University of Edinburgh,=27,0.920000
4635,GB,United Kingdom,=23,Europe,86.9,None,King's College London,3650.0,23780.0,1513.0,9451.0,King’s College London,United Kingdom,King’s College London,36,0.952381


In [153]:
#do we have to add sth manually??
#df_cartesian[(df_cartesian.similar<0.87) & (df_cartesian.similar>=0.80)  & (!df_cartesian.title.str.startswith('University'))]

# Task 4 - Exploratory Analysis 

# Task 5 - Finding the best university 